In [1]:
import numpy as np
from stable_baselines3 import PPO
# from sb3_contrib import TRPO
from gymnasium import spaces, Env
from boat_simulation import Boat, wrap_phase

2024-02-09 15:34:30.740819: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-09 15:34:30.759033: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-09 15:34:30.759047: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-09 15:34:30.759518: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-09 15:34:30.762960: I tensorflow/core/platform/cpu_feature_guar

In [2]:
import torch
import torch.nn as nn
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor


def linear(device, x, in_units, out_units, activation):
    x = nn.Linear(in_units, out_units).to(device)(x)
    return activation().to(device)(x)

def mlp(device, x, units):
    for i in range(0, len(units)):
        if not isinstance(units[i], tuple):
            units[i] = (units[i], nn.ReLU)

    for i in range(1, len(units)):
        prev_layer_size, _ = units[i - 1]
        layer_size, activation = units[i]
        x = linear(device, x, prev_layer_size, layer_size, activation)
    return x

def self_attention(device, x, num_heads, num_layers):
    _batch, seq_len, embed_size = x.shape

    for i in range(1, num_layers):
        lin = linear(device, x, embed_size, embed_size, nn.ReLU)
        attn,  _weights = nn.MultiheadAttention(embed_size, num_heads, dropout=0.5).to(device)(lin, lin, lin, need_weights=False)
        x = attn + x
    return x


class CustomMLP(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=1):
        super(CustomMLP, self).__init__(observation_space, features_dim)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def forward(self, observations):
        x = observations.to(self.device)
        return mlp(self.device, x, [7, 400, 300, (self.features_dim, nn.Tanh)])


# Define the LSTM model
class NonLSTM(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim):
        super(NonLSTM, self).__init__(observation_space, features_dim)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.hidden_layer_size = 64
        x1, x2 = observation_space.shape
        input_size = x1 * x2
        self.linear = lambda x: mlp(self.device, x, [input_size, self.hidden_layer_size, self.hidden_layer_size, features_dim])

    def forward(self, input_seq):
        batch_size = input_seq.shape[0]
        input = input_seq.view(batch_size, -1)
        predictions = self.linear(input)
        return predictions

class SelfAttentionExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim):
        super(SelfAttentionExtractor, self).__init__(observation_space, features_dim)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        (seq_size, input_size,) = observation_space.shape
        self.attention = lambda x: self_attention(self.device, x, 8, 1)
        self.input_layer = lambda x: mlp(self.device, x, [input_size, features_dim])
        self.output_layer = lambda x: mlp(self.device, x, [seq_size * features_dim, (seq_size // 2 + 1) * features_dim, features_dim])

    def forward(self, input_seq):
        input = self.input_layer(input_seq.to(self.device))
        hidden = self.attention(input)
        hidden_flat = nn.Flatten().to(self.device)(hidden)
        output = self.output_layer(hidden_flat)
        return output

In [3]:
from environments import MultiMarkEnv

In [4]:
from IPython.core.display import display, HTML, Javascript

disable_scroll_script = """
var idx = Jupyter.notebook.get_selected_index();
var cell = Jupyter.notebook.get_cell(idx);
cell.output_area._should_scroll = function(lines) {
    return false;
}
"""

display(Javascript(disable_scroll_script))

/tmp/ipykernel_65087/738611061.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML, Javascript


<IPython.core.display.Javascript object>

In [5]:
import plotly.graph_objects as go
import time
import ipywidgets as widgets
from IPython.display import display

outer_radius = 2 * 250 + 2.5 * 0.1 * 250

dt = 15
num_marks = 2
max_seconds = 500 * num_marks / dt

# Initialize the figure and scatter plot
fig = go.FigureWidget()
scatter = fig.add_scatter(mode='markers+lines', name='trajectory')
marks_scatter = fig.add_scatter(mode='markers+text', name='marks')
fig.update_xaxes(range=[-outer_radius, outer_radius],dtick=25)
fig.update_yaxes(range=[-outer_radius, outer_radius],dtick=25)
fig.layout.width=800
fig.layout.height=800

heading_fig = go.FigureWidget()
heading_scatter = heading_fig.add_scatter(mode='markers+lines', name='heading')
rudder_angle_scatter = heading_fig.add_scatter(mode='markers+lines', name='rudder angle')
heading_fig.update_xaxes(range=[0, max_seconds],dtick=25)
heading_fig.update_yaxes(range=[-210, 210],dtick=30)
heading_fig.layout.width=800
heading_fig.layout.height=600
heading_fig.layout.title = "Heading over time"

vmg_fig = go.FigureWidget()
vmg_scatter = vmg_fig.add_scatter(mode='markers+lines')
vmg_fig.update_xaxes(range=[0, max_seconds],dtick=25)
vmg_fig.update_yaxes(range=[-4, 4],dtick=2)
vmg_fig.layout.width=800
vmg_fig.layout.height=400
vmg_fig.layout.title = "VMG over time"

# Initialize output for the text
out = widgets.Output()

# Display text and figure
display(out)
display(fig)
display(heading_fig)
display(vmg_fig)

episode = 0

colormap = ['blue', 'red', 'green', 'orange', 'black']

def plot(data, marks, bounds):
    global episode, out, fig
    episode += 1
    if episode % 50 != 0:
        return
    x_values = [point['x'] for point in data]
    y_values = [point['y'] for point in data]
    meta_values = [{k: '%.3f' % v for (k, v) in point['meta'].items()} for point in data]

    min_x, max_x, min_y, max_y = bounds


    color_meta = [point['meta']['current_mark'] for point in data]
    colors = [colormap[color % len(colormap)] for color in color_meta]

    headings = [point['meta']['heading'] for point in data]
    rudder_angles = [point['meta']['rudder_angle'] for point in data]
    vmg = [point['meta']['vmg'] for point in data]

    global_meta = {'final_position': (x_values[-1], y_values[-1]), 'reward': sum([point['meta']['reward'] for point in data]), 'iters': len(data), 'episode': episode}

    # Update data
    scatter = fig.data[0]
    scatter.x = x_values
    scatter.y = y_values
    scatter.hovertext = meta_values  # Add this line
    scatter.marker.color = colors
    scatter.line.color = 'lightgrey'

    marks_scatter = fig.data[1]
    marks_scatter.x = [x for x, _ in marks]
    marks_scatter.y = [y for _, y in marks]
    marks_scatter.text = list(range(len(marks)))
    marks_scatter.textposition = 'bottom right'
    marks_scatter.marker.color = 'black'
    marks_scatter.marker.symbol = 'x'
    marks_scatter.marker.size = 10

    heading_scatter = heading_fig.data[0]
    heading_scatter.x = list(range(len(headings)))
    heading_scatter.y = headings
    heading_scatter.marker.color = colors
    heading_scatter.line.color = 'lightgrey'

    rudder_angle_scatter = heading_fig.data[1]
    rudder_angle_scatter.x = list(range(len(rudder_angles)))
    rudder_angle_scatter.y = rudder_angles
    rudder_angle_scatter.marker.color = 'lightgreen'
    rudder_angle_scatter.line.color = 'lightgreen'

    vmg_scatter = vmg_fig.data[0]
    vmg_scatter.x = list(range(len(vmg)))
    vmg_scatter.y = vmg
    vmg_scatter.marker.color = colors
    vmg_scatter.line.color = 'lightgrey'

    fig.update_xaxes(range=[min_x * 1.1, max_x * 1.1],dtick=25)
    fig.update_yaxes(range=[min_y * 1.1, max_y * 1.1],dtick=25)
    fig.layout.width=800
    fig.layout.height=800

    heading_fig.update_yaxes(range=[-210, 210],dtick=30)
    heading_fig.layout.width=800
    heading_fig.layout.height=600

    vmg_fig.layout.width=800
    vmg_fig.layout.height=400

    # Update the text output
    with out:
        out.clear_output(wait=True)
        print(f"Global Meta: {str(global_meta)}")

None

Output()

FigureWidget({
    'data': [{'mode': 'markers+lines',
              'name': 'trajectory',
              'type': 'scatter',
              'uid': 'da5863f0-5477-4cc0-b3fe-b6a9f342159c'},
             {'mode': 'markers+text', 'name': 'marks', 'type': 'scatter', 'uid': 'f00fb6fa-328e-4628-9fb6-0ce990fa56f5'}],
    'layout': {'height': 800,
               'template': '...',
               'width': 800,
               'xaxis': {'dtick': 25, 'range': [-562.5, 562.5]},
               'yaxis': {'dtick': 25, 'range': [-562.5, 562.5]}}
})

FigureWidget({
    'data': [{'mode': 'markers+lines',
              'name': 'heading',
              'type': 'scatter',
              'uid': '6f4d3bef-39d0-4f0f-8c1e-3ddf62cdf236'},
             {'mode': 'markers+lines',
              'name': 'rudder angle',
              'type': 'scatter',
              'uid': '52fe2cfe-e5fe-4bd6-9108-3b917eadfd02'}],
    'layout': {'height': 600,
               'template': '...',
               'title': {'text': 'Heading over time'},
               'width': 800,
               'xaxis': {'dtick': 25, 'range': [0, 66.66666666666667]},
               'yaxis': {'dtick': 30, 'range': [-210, 210]}}
})

FigureWidget({
    'data': [{'mode': 'markers+lines', 'type': 'scatter', 'uid': 'a816f300-2b3a-4420-86dd-a899df3749d8'}],
    'layout': {'height': 400,
               'template': '...',
               'title': {'text': 'VMG over time'},
               'width': 800,
               'xaxis': {'dtick': 25, 'range': [0, 66.66666666666667]},
               'yaxis': {'dtick': 2, 'range': [-4, 4]}}
})

In [6]:
# Configuration

r = 250
actions = np.array([-90, -45, 0, 45, 90]) / 180.0

config = {
    'max_marks': 1,
    'max_seconds_per_leg': 500,
    'plot_fn': plot,
    'leg_radius': r,
    'actions': actions,
    'target_tolerance_multiplier': 5
}

outer_radius = config['max_marks'] * 2 * r + 0.2 * r
bounds = [-outer_radius, outer_radius, -outer_radius, outer_radius]
# Initialize Environment
env = MultiMarkEnv(config, dt=15, bounds=bounds, seq_size=1, phase_steps=8)

# Initialize PPO model
model = PPO("MlpPolicy", env, verbose=1, device='cuda', policy_kwargs={
    "net_arch": [64, 400, 300],
    # "features_extractor_class": SelfAttentionExtractor,
    # "features_extractor_kwargs": {"features_dim": 64}
})

# Train the model
# model.learn(total_timesteps=1_000_000)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [7]:
model.learn(total_timesteps=500_000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 30.5     |
|    ep_rew_mean     | -1.33    |
| time/              |          |
|    fps             | 1038     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 30.1        |
|    ep_rew_mean          | -0.335      |
| time/                   |             |
|    fps                  | 975         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.017316323 |
|    clip_fraction        | 0.291       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.59       |
|    explained_variance   | -0.00339    |
|    learning_rate        | 0.

In [8]:
# Initialize Environment
env = MultiMarkEnv(config, dt=15, bounds=bounds, seq_size=1, phase_steps=16)

# Initialize PPO model
model = PPO("MlpPolicy", env, verbose=1, device='cuda', policy_kwargs={
    "net_arch": [64, 400, 300],
    # "features_extractor_class": SelfAttentionExtractor,
    # "features_extractor_kwargs": {"features_dim": 64}
})

model.learn(total_timesteps=1_000_000)


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 29.4     |
|    ep_rew_mean     | -0.869   |
| time/              |          |
|    fps             | 1236     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 29.2       |
|    ep_rew_mean          | -0.367     |
| time/                   |            |
|    fps                  | 1058       |
|    iterations           | 2          |
|    time_elapsed         | 3          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01789459 |
|    clip_fraction        | 0.34       |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.5

In [9]:
model.save("BoatControl_boat_model_coarse.dat")

In [10]:
import plotly.graph_objects as go
import time
import ipywidgets as widgets
from IPython.display import display

outer_radius = 2 * 250 + 2.5 * 0.1 * 250

dt = 5
num_marks = 2
max_seconds = 500 * num_marks / dt

# Initialize the figure and scatter plot
fig = go.FigureWidget()
scatter = fig.add_scatter(mode='markers+lines', name='trajectory')
marks_scatter = fig.add_scatter(mode='markers+text', name='marks')
fig.update_xaxes(range=[-outer_radius, outer_radius],dtick=25)
fig.update_yaxes(range=[-outer_radius, outer_radius],dtick=25)
fig.layout.width=800
fig.layout.height=800

heading_fig = go.FigureWidget()
heading_scatter = heading_fig.add_scatter(mode='markers+lines', name='heading')
rudder_angle_scatter = heading_fig.add_scatter(mode='markers+lines', name='rudder angle')
heading_fig.update_xaxes(range=[0, max_seconds],dtick=25)
heading_fig.update_yaxes(range=[-210, 210],dtick=30)
heading_fig.layout.width=800
heading_fig.layout.height=600
heading_fig.layout.title = "Heading over time"

vmg_fig = go.FigureWidget()
vmg_scatter = vmg_fig.add_scatter(mode='markers+lines')
vmg_fig.update_xaxes(range=[0, max_seconds],dtick=25)
vmg_fig.update_yaxes(range=[-4, 4],dtick=2)
vmg_fig.layout.width=800
vmg_fig.layout.height=400
vmg_fig.layout.title = "VMG over time"

# Initialize output for the text
out = widgets.Output()

# Display text and figure
display(out)
display(fig)
display(heading_fig)
display(vmg_fig)

episode = 0

colormap = ['blue', 'red', 'green', 'orange', 'black']

def plot_new(data, marks, bounds):
    global episode, out, fig
    episode += 1
    if episode % 50 != 0:
        return
    x_values = [point['x'] for point in data]
    y_values = [point['y'] for point in data]
    meta_values = [{k: '%.3f' % v for (k, v) in point['meta'].items()} for point in data]

    min_x, max_x, min_y, max_y = bounds


    color_meta = [point['meta']['current_mark'] for point in data]
    colors = [colormap[color % len(colormap)] for color in color_meta]

    headings = [point['meta']['heading'] for point in data]
    rudder_angles = [point['meta']['rudder_angle'] for point in data]
    vmg = [point['meta']['vmg'] for point in data]

    global_meta = {'final_position': (x_values[-1], y_values[-1]), 'reward': sum([point['meta']['reward'] for point in data]), 'iters': len(data), 'episode': episode}

    # Update data
    scatter = fig.data[0]
    scatter.x = x_values
    scatter.y = y_values
    scatter.hovertext = meta_values  # Add this line
    scatter.marker.color = colors
    scatter.line.color = 'lightgrey'

    marks_scatter = fig.data[1]
    marks_scatter.x = [x for x, _ in marks]
    marks_scatter.y = [y for _, y in marks]
    marks_scatter.text = list(range(len(marks)))
    marks_scatter.textposition = 'bottom right'
    marks_scatter.marker.color = 'black'
    marks_scatter.marker.symbol = 'x'
    marks_scatter.marker.size = 10

    heading_scatter = heading_fig.data[0]
    heading_scatter.x = list(range(len(headings)))
    heading_scatter.y = headings
    heading_scatter.marker.color = colors
    heading_scatter.line.color = 'lightgrey'

    rudder_angle_scatter = heading_fig.data[1]
    rudder_angle_scatter.x = list(range(len(rudder_angles)))
    rudder_angle_scatter.y = rudder_angles
    rudder_angle_scatter.marker.color = 'lightgreen'
    rudder_angle_scatter.line.color = 'lightgreen'

    vmg_scatter = vmg_fig.data[0]
    vmg_scatter.x = list(range(len(vmg)))
    vmg_scatter.y = vmg
    vmg_scatter.marker.color = colors
    vmg_scatter.line.color = 'lightgrey'

    fig.update_xaxes(range=[min_x * 1.1, max_x * 1.1],dtick=25)
    fig.update_yaxes(range=[min_y * 1.1, max_y * 1.1],dtick=25)
    fig.layout.width=800
    fig.layout.height=800

    heading_fig.update_yaxes(range=[-210, 210],dtick=30)
    heading_fig.layout.width=800
    heading_fig.layout.height=600

    vmg_fig.layout.width=800
    vmg_fig.layout.height=400

    # Update the text output
    with out:
        out.clear_output(wait=True)
        print(f"Global Meta: {str(global_meta)}")

None

Output()

FigureWidget({
    'data': [{'mode': 'markers+lines',
              'name': 'trajectory',
              'type': 'scatter',
              'uid': 'dffdb400-74d6-4edf-b975-af4efc676476'},
             {'mode': 'markers+text', 'name': 'marks', 'type': 'scatter', 'uid': 'fb70e767-7f02-48d1-8cb3-25e6ec15fbe9'}],
    'layout': {'height': 800,
               'template': '...',
               'width': 800,
               'xaxis': {'dtick': 25, 'range': [-562.5, 562.5]},
               'yaxis': {'dtick': 25, 'range': [-562.5, 562.5]}}
})

FigureWidget({
    'data': [{'mode': 'markers+lines',
              'name': 'heading',
              'type': 'scatter',
              'uid': '97407725-eed9-49f9-bc87-25d7d07f8065'},
             {'mode': 'markers+lines',
              'name': 'rudder angle',
              'type': 'scatter',
              'uid': '7fb6bba3-0adb-4afe-ae1b-ee7da544fb93'}],
    'layout': {'height': 600,
               'template': '...',
               'title': {'text': 'Heading over time'},
               'width': 800,
               'xaxis': {'dtick': 25, 'range': [0, 200.0]},
               'yaxis': {'dtick': 30, 'range': [-210, 210]}}
})

FigureWidget({
    'data': [{'mode': 'markers+lines', 'type': 'scatter', 'uid': '7a4d26d1-154b-40a1-b844-109793e0631f'}],
    'layout': {'height': 400,
               'template': '...',
               'title': {'text': 'VMG over time'},
               'width': 800,
               'xaxis': {'dtick': 25, 'range': [0, 200.0]},
               'yaxis': {'dtick': 2, 'range': [-4, 4]}}
})

In [11]:
config_new = {**config}
config_new['plot_fn'] = plot_new
config_new['target_tolerance_multiplier'] = 3

model.set_env(MultiMarkEnv(config, dt=10, seq_size=1, bounds=bounds, phase_steps=16))

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [12]:
model.learn(total_timesteps=1500_000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 21.5     |
|    ep_rew_mean     | 3.64     |
| time/              |          |
|    fps             | 1302     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 20.5        |
|    ep_rew_mean          | 3.6         |
| time/                   |             |
|    fps                  | 1117        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.056450576 |
|    clip_fraction        | 0.232       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.461      |
|    explained_variance   | 0.853       |
|    learning_rate        | 0.

In [13]:
model.save("BoatControl_boat_model_level_2.dat")

In [14]:
config_new = {**config}
config_new['plot_fn'] = plot_new
config_new['target_tolerance_multiplier'] = 2

model.set_env(MultiMarkEnv(config, dt=5, seq_size=1, bounds=bounds, phase_steps=16))

model.learn(total_timesteps=1500_000)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 37.9     |
|    ep_rew_mean     | 6.97     |
| time/              |          |
|    fps             | 1706     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 33.7        |
|    ep_rew_mean          | 6.78        |
| time/                   |             |
|    fps                  | 1426        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.047110356 |
|    clip_fraction        | 0.229       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.367      |
|    explained_variance   | 0.753       |
|    learning_rate        | 0.

In [15]:
model.save("BoatControl_boat_model_level_3.dat")

In [16]:
config_new = {**config}
config_new['plot_fn'] = plot_new
config_new['target_tolerance_multiplier'] = 1.5

model.set_env(MultiMarkEnv(config, dt=3, seq_size=1, bounds=bounds, phase_steps=20))

model.learn(total_timesteps=1_500_000)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 51       |
|    ep_rew_mean     | 9.86     |
| time/              |          |
|    fps             | 1781     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 52.8      |
|    ep_rew_mean          | 10.8      |
| time/                   |           |
|    fps                  | 1483      |
|    iterations           | 2         |
|    time_elapsed         | 2         |
|    total_timesteps      | 4096      |
| train/                  |           |
|    approx_kl            | 0.0615698 |
|    clip_fraction        | 0.196     |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.383    |
|    explained_variance   | 0.623     |
|    learning_rate        | 0.0003    |
|    loss           

In [17]:
model.save("BoatControl_boat_model_fine.dat")


In [18]:
env = MultiMarkEnv(config, dt=5, bounds=bounds, seq_size=1)


def render(data, marks, bounds):
    x_values = [point['x'] for point in data]
    y_values = [point['y'] for point in data]
    meta_values = [{k: '%.3f' % v for (k, v) in point['meta'].items()} for point in data]

    min_x, max_x, min_y, max_y = bounds


    color_meta = [point['meta']['current_mark'] for point in data]
    colors = [colormap[color % len(colormap)] for color in color_meta]

    headings = [point['meta']['heading'] for point in data]
    rudder_angles = [point['meta']['rudder_angle'] for point in data]
    vmg = [point['meta']['vmg'] for point in data]

    global_meta = {'final_position': (x_values[-1], y_values[-1]), 'reward': sum([point['meta']['reward'] for point in data]), 'iters': len(data), 'episode': episode}

    # Update data
    scatter = fig.data[0]
    scatter.x = x_values
    scatter.y = y_values
    scatter.hovertext = meta_values  # Add this line
    scatter.marker.color = colors
    scatter.line.color = 'lightgrey'

    marks_scatter = fig.data[1]
    marks_scatter.x = [x for x, _ in marks]
    marks_scatter.y = [y for _, y in marks]
    marks_scatter.text = list(range(len(marks)))
    marks_scatter.textposition = 'bottom right'
    marks_scatter.marker.color = 'black'
    marks_scatter.marker.symbol = 'x'
    marks_scatter.marker.size = 10

    heading_scatter = heading_fig.data[0]
    heading_scatter.x = list(range(len(headings)))
    heading_scatter.y = headings
    heading_scatter.marker.color = colors
    heading_scatter.line.color = 'lightgrey'

    rudder_angle_scatter = heading_fig.data[1]
    rudder_angle_scatter.x = list(range(len(rudder_angles)))
    rudder_angle_scatter.y = rudder_angles
    rudder_angle_scatter.marker.color = 'lightgreen'
    rudder_angle_scatter.line.color = 'lightgreen'

    vmg_scatter = vmg_fig.data[0]
    vmg_scatter.x = list(range(len(vmg)))
    vmg_scatter.y = vmg
    vmg_scatter.marker.color = colors
    vmg_scatter.line.color = 'lightgrey'

    fig.update_xaxes(range=[min_x * 1.1, max_x * 1.1],dtick=25)
    fig.update_yaxes(range=[min_y * 1.1, max_y * 1.1],dtick=25)
    fig.layout.width=800
    fig.layout.height=800

    heading_fig.update_yaxes(range=[-210, 210],dtick=30)
    heading_fig.layout.width=800
    heading_fig.layout.height=600

    vmg_fig.layout.width=800
    vmg_fig.layout.height=400

    # Update the text output
    with out:
        out.clear_output(wait=True)
        print(f"Global Meta: {str(global_meta)}")


# Initialize output for the text
out = widgets.Output()

# Display text and figure
display(out)
display(fig)
display(heading_fig)
display(vmg_fig)

Output()

FigureWidget({
    'data': [{'hovertext': [{'current_mark': '0.000', 'vmg': '0.000', 'heading':
                            '0.000', 'rudder_angle': '45.000', 'reward': '0.000',
                            'speed': '0.000'}, {'current_mark': '0.000', 'vmg':
                            '-0.000', 'heading': '2.185', 'rudder_angle': '45.000',
                            'reward': '-0.000', 'speed': '0.000'}, {'current_mark':
                            '0.000', 'vmg': '-0.000', 'heading': '6.555',
                            'rudder_angle': '90.000', 'reward': '-0.000', 'speed':
                            '0.000'}, {'current_mark': '0.000', 'vmg': '-0.000',
                            'heading': '10.925', 'rudder_angle': '90.000',
                            'reward': '-0.000', 'speed': '0.000'}, {'current_mark':
                            '0.000', 'vmg': '-0.000', 'heading': '15.295',
                            'rudder_angle': '90.000', 'reward': '-0.000', 'speed':
                   

FigureWidget({
    'data': [{'line': {'color': 'lightgrey'},
              'marker': {'color': [blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, red]},
              'mode': 'markers+lines',
              'name': 'heading',
              'type': 'scatter',
              'uid': '97407725-eed9-49f9-bc87-25d7d07f8065',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
                    34],
              'y': [0.0, 2.1849784199661966, 6.55493525989859, 10.924892099830984,
                    15.294848939763376, 19.66480577969577, 24.034762619628165,
                    28.40471945956

FigureWidget({
    'data': [{'line': {'color': 'lightgrey'},
              'marker': {'color': [blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, red]},
              'mode': 'markers+lines',
              'type': 'scatter',
              'uid': '7a4d26d1-154b-40a1-b844-109793e0631f',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
                    34],
              'y': [0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0,
                    -0.13926294625365265, 0.4067497721611348, 1.0439559203367235,
                    1.6741791921520297, 2.2557349469063994, 2.7393802459415526,
  

In [19]:
target_x = 0
target_y = 250

best_trajectory = None
best_reward = None
for i in range(20):
    obs, _ = env.reset()
    env.target_x[0] = target_x
    env.target_y[0] = target_y
    env.boat.heading = np.pi / 4
    env.heading = np.pi/4
    done = False
    total_reward = 0
    while not done:
        action, _states = model.predict(obs)
        obs, reward, is_terminal, is_truncated, info = env.step(action)
        done = is_terminal or is_truncated
        total_reward += reward

    if best_reward is None or total_reward > best_reward:
        best_reward = total_reward
        best_trajectory = env.trajectory

render(best_trajectory, [(target_x, target_y)], bounds)